# MECH 309: Assignment 2, Question 3

Cagri Arslan

January 30, 2025

*All work can be found on https://github.com/imported-canuck/MECH-309*

In [56]:
# Imports
import numpy as np 
from scipy import linalg
import solver 

*Note:* I wrote all scripts by hand. Therefore, code might not be as concise/clean as a script produced by ChatGPT or taken from StackOverflow. Regardless, scripts should still be fully functional if all **assumptions** stated in docstrings are respected. I trust that this won't be penalized during grading. 

In [57]:
def forward_sub(A, b):
    """
    Solve the equation Ax = b for x, where A is a lower triangular matrix. Assumes a lower 
    triangular matrix A. Expects a that A is non-singular, and that the dimensions of 
    A and b are compatible (but checks these and throws an exception if not).
    
    Parameters:
    A (ndarray): A lower triangular matrix of shape (n, n).
    b (ndarray): A vector of shape (n, 1).
    
    Returns:
    x (ndarray): The solution vector of shape (n, 1).
    """
    n = A.shape[0]           # Size of matrix A
    if A.shape[1] != n:      # Check if A is square
        raise ValueError("Matrix A must be square.")
    if b.shape[0] != n:      # Check if dimensions of b are compatible
        raise ValueError("Vector b must have compatible dimensions with matrix A.")

    x = np.zeros((n, 1))           # Initialize solution vector x as a column vector
    
    x[0, 0] = b[0, 0] / A[0, 0]    # First element of x (actually redundant, kept for clarity)
    
    for i in range(n):       # Loop over each row
        if A[i, i] == 0:     # Check for singularity (zero diagonal element)
            raise ValueError("Matrix is singular.")
        
        # For each column of row i before the diagonal subtract the product of 
        # the entry A(i, j) and the corresponding entry of vector x from b[i]
        for j in range(i):
            b[i, 0] -= A[i, j] * x[j, 0] 

        # Finaly divide b[i] by the diagonal element A(i, i) to get x[i]
        x[i, 0] = b[i, 0] / A[i, i]  

    return x 


In [58]:
def backward_sub(A, b):
    """
    Solve the equation Ax = b for x, where A is an upper triangular matrix. Assumes an upper 
    triangular matrix A. Expects a that A is non-singular, and that the dimensions of 
    A and b are compatible (but checks these and throws an exception if not).
    
    Parameters:
    A (ndarray): An upper triangular matrix of shape (n, n).
    b (ndarray): A vector of shape (n, 1).
    
    Returns:
    x (ndarray): The solution vector of shape (n, 1).
    """
    n = A.shape[0]               # Size of matrix A
    if A.shape[1] != n:          # Check if A is square
        raise ValueError("Matrix A must be square.")
    if b.shape[0] != n:          # Check if dimensions of b are compatible
        raise ValueError("Vector b must have compatible dimensions with matrix A.")

    x = np.zeros((n, 1))              # Initialize solution vector x
    
    x[-1, 0] = b[-1, 0] / A[-1, -1]    # First element of x (bottom-right; actually redundant, kept for clarity)
    
    for i in range(n - 1, -1 , -1):    # Loop over each row from bottom to top
        if A[i, i] == 0:               # Check for singularity (zero diagonal element)
            raise ValueError("Matrix is singular.")

        # For each column of row i after the diagonal subtract the product of 
        # the entry A(i, j) and the corresponding entry of vector x from b[i]    
        for j in range(i + 1, n):
            b[i, 0] -= A[i, j] * x[j, 0]

        # Finaly divide b[i] by the diagonal element A(i, i) to get x[i]
        x[i, 0] = b[i, 0] / A[i, i]
           
    return x


In [ ]:
def gaussian_elimination(A, b):
    """
    Solve the equation Ax = b for x using Gaussian elimination with partial pivoting. 
    Expects a that A is non-singular, and that the dimensions of A and b are 
    compatible (but checks these and throws an exception if not).
    
    Parameters:
    A (ndarray): An matrix of shape (n, n).
    b (ndarray): A vector of shape (n, 1).
    
    Returns:
    x (ndarray): The solution vector of shape (n, 1).
    """    
    tol = 1e-12                  # Tolerance to avoid floating point issues
    n = A.shape[0]               # Size of matrix A
    
    if A.shape[1] != n:          # Check if A is square
        raise ValueError("Matrix A must be square.")
    if b.shape[0] != n:          # Check if dimensions of b are compatible
        raise ValueError("Vector b must have compatible dimensions with matrix A.")

    for i in range(n - 1):

        p = i + np.argmax(np.abs(A[i:, i]))

        if np.abs(A[p, i]) <= tol:
                raise ValueError("Matrix is singular.")
            
        if p != i:
                A[[i, p], :] = A[[p, i], :]
                b[[i, p], :] = b[[p, i], :]

        for j in range(i + 1, n): 
            factor = A[j, i] / A[i, i] 
            A[j] = A[j] - factor * A[i]
            b[j] = b[j] - factor * b[i]

    return backward_sub(A, b)


[[  0.   1.]
 [ nan -inf]]
[[ 0.]
 [nan]]


C:\Users\cagri\AppData\Local\Temp\ipykernel_21168\3799556440.py:24: RuntimeWarning: divide by zero encountered in scalar divide
  factor = A[j, i] / A[i, i]
C:\Users\cagri\AppData\Local\Temp\ipykernel_21168\3799556440.py:25: RuntimeWarning: invalid value encountered in multiply
  A[j] = A[j] - factor * A[i]
C:\Users\cagri\AppData\Local\Temp\ipykernel_21168\3799556440.py:26: RuntimeWarning: invalid value encountered in multiply
  b[j] = b[j] - factor * b[i]


ValueError: Matrix is singular.

In [ ]:
def LU_factorization(A, b):
    """
    Solve the equation Ax = b for x using LU factorization with partial pivoting. 
    Expects a that A is non-singular, and that the dimensions of A and b are 
    compatible (but checks these and throws an exception if not).
    
    Parameters:
    A (ndarray): An matrix of shape (n, n).
    b (ndarray): A vector of shape (n, 1).
    
    Returns:
    x (ndarray): The solution vector of shape (n, 1).
    L (ndarray): The lower triangular matrix of shape (n, n).
    U (ndarray): The upper triangular matrix of shape (n, n).
    """    
    tol = 1e-12                  # Tolerance to avoid floating point issues
    n = A.shape[0]               # Size of matrix A

    if A.shape[1] != n:          # Check if A is square
        raise ValueError("Matrix A must be square.")
    if b.shape[0] != n:          # Check if dimensions of b are compatible
        raise ValueError("Vector b must have compatible dimensions with matrix A.")

    L = np.eye(n)               # Initialize L as identity matrix

    for i in range(n - 1):

        p = i + np.argmax(np.abs(A[i:, i]))

        if np.abs(A[p, i]) <= tol:
                raise ValueError("Matrix is singular.")
            
        if p != i:
                A[[i, p], :] = A[[p, i], :]
                b[[i, p], :] = b[[p, i], :]

        for j in range(i + 1, n): 
            factor = A[j, i] / A[i, i] 
            A[j] = A[j] - factor * A[i]
            b[j] = b[j] - factor * b[i]

            ## FINISH L MATRIX ##

    U = A.copy()

    y = forward_sub(L, b)
    x = backward_sub(U, y)

    return L, U, x


(array([[1. , 0. , 0. ],
        [3. , 1. , 0. ],
        [5. , 2.5, 1. ]]),
 array([[  1.,   3.,   5.],
        [  0.,  -4., -10.],
        [  0.,   0.,   5.]]),
 array([[1.],
        [1.],
        [1.]]))